In [13]:
import pandas as pd
import hashlib
import os

# Get the current working directory
current_directory = os.getcwd()
# Define the path to the CSV file
processed_data_path = os.path.join(current_directory, '..', '..', 'data', 'processed')
raw_data_path = os.path.join(current_directory, '..', '..', 'data', 'raw', 'DataCoSupplyChainDataset.csv')

df = pd.read_csv(raw_data_path, encoding='ISO-8859-1')

# Partitioning the DataFrame
dim_customer = df[[
    'Customer Id', # primary key
    'Customer Fname', 
    'Customer Lname', 
    'Customer Email', 
    'Customer Country',
    'Latitude',
    'Longitude',
    'Customer City', 
    'Customer State', 
    'Customer Street', 
    'Customer Zipcode', 
    'Customer Segment'
]]

dim_product = df[[
    'Product Card Id', # primary key
    'Department Id',
    'Department Name',
    'Product Category Id',
    'Category Name',
    'Product Name', 
    'Product Description', 
    'Product Price', 
    'Product Status', 
]]

dim_region = df[[
    'Market',
    'Order Region',
    'Order Country',
    'Order State',
    'Order City'
]].drop_duplicates()

dim_shipping = df[[
    'Days for shipment (scheduled)', 
    'Shipping Mode'
]]

Fact_Sales = df[[
    'Customer Id', 
    'Product Card Id',
    'Category Id',
    'Order Id', 
    'Department Id',
    'Sales', 
    'Order Item Quantity',
    'Market',
    'Order Region',
    'Order Country',
    'Order State',
    'Order City',
    'Benefit per order',
    'Order Item Cardprod Id',
    'Order Item Discount',
    'Order Item Product Price',
    'Order Item Discount Rate',
    'Order Item Profit Ratio',
    'Order Profit Per Order',
    "Type",
    'Order Status',
    'Order Zipcode',
    'order date (DateOrders)',
    'Days for shipping (real)',
    'shipping date (DateOrders)',
    'Delivery Status',
    'Late_delivery_risk',
    'Days for shipment (scheduled)', 
    'Shipping Mode'
]]

date_columns = ['order date (DateOrders)', 'shipping date (DateOrders)']

cols_to_drop = [
    'Market',
    'Order Region',
    'Order Country',
    'Order State',
    'Order City',
    'Days for shipment (scheduled)', 
    'Shipping Mode'
]

new_cols_order = [
    'Customer Id',
    'Product Card Id',
    'Order Id',
    'Region Id',
    'Sales',
    'Order Item Quantity',
    'Benefit per order',
    'Order Item Cardprod Id',
    'Order Item Discount',
    'Order Item Discount Rate',
    'Order Item Product Price',
    'Order Item Profit Ratio',
    'Order Profit Per Order',
    'Type',
    'Order Status',
    'Order Zipcode',
    'order date (DateOrders)',
    'Days for shipping (real)',
    'Shipping Method Id',
    'shipping date (DateOrders)',
    'Delivery Status',
    'Late_delivery_risk'
]

def generate_region_id(region_df):
    """
    Generates a unique numeric Region Id based on the combination of 
    Order Region, Order Country, Order State, and Order City.
    
    Args:
    region_df (pd.DataFrame): DataFrame containing the region information.

    Returns:
    pd.Series: A series containing the unique numeric Region Ids.
    """
    # Create a combined string for each row
    combined_strings = (
        region_df['Market'].astype(str) + "_" +
        region_df['Order Region'].astype(str) + "_" +
        region_df['Order Country'].astype(str) + "_" +
        region_df['Order State'].astype(str) + "_" +
        region_df['Order City'].astype(str)
    )
    
    # Generate a consistent hash using hashlib (e.g., SHA-256)
    def consistent_hash(value):
        # Convert to a SHA-256 hash and take the first 10 digits as an integer
        return int(hashlib.sha256(value.encode()).hexdigest(), 16) % (10 ** 10)
    
    # Apply the consistent hash function to each combined string
    region_ids = combined_strings.apply(consistent_hash)
    
    return region_ids

def create_date_dimension(df, date_columns):
    """
    Creates a date dimension DataFrame with a range of dates between the earliest
    and latest date in the specified date columns. The output DataFrame includes
    datekey, date, year, quarter, month, and weekday.

    Args:
    df (pd.DataFrame): Input DataFrame containing the date columns.
    date_columns (list): List of date columns to consider for the range.

    Returns:
    pd.DataFrame: A date dimension DataFrame.
    """
    # Create a copy of df to avoid modifying the original DataFrame directly
    df = df.copy()

    # Convert the specified date columns to datetime
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

    # Get the earliest and latest dates across the specified columns
    earliest_date = df[date_columns].min().min()
    latest_date = df[date_columns].max().max()

    # Generate a date range from earliest to latest date
    date_range = pd.date_range(start=earliest_date, end=latest_date)

    # Create a DataFrame with the date range
    date_df = pd.DataFrame({
        'date': date_range
    })

    # Add additional columns
    date_df['datekey'] = date_df['date'].dt.strftime('%m%d%Y').astype(int)  # Date key in mmddyyyy format
    date_df['year'] = date_df['date'].dt.year
    date_df['quarter'] = date_df['date'].dt.quarter
    date_df['month'] = date_df['date'].dt.month
    date_df['weekday'] = date_df['date'].dt.weekday  # Monday = 0, Sunday = 6
    date_df['weekday_name'] = date_df['date'].dt.day_name() # Add weekday name as a string

    return date_df

def generate_shipping_method_id(df):
    """
    Generates a unique numeric Shipping Method Id based on the combination of 
    'Days for shipment (scheduled)' and 'Shipping Mode' using a consistent hash function.
    
    Args:
    df (pd.DataFrame): DataFrame containing the shipping information.

    Returns:
    pd.Series: A series containing the unique numeric Shipping Method Ids.
    """
    # Create a combined string for each row
    combined_strings = (
        df['Days for shipment (scheduled)'].astype(str) + "_" +
        df['Shipping Mode'].astype(str)
    )
    
    # Generate a consistent hash using hashlib (e.g., SHA-256)
    def consistent_hash(value):
        # Convert to a SHA-256 hash and take the first 10 digits as an integer
        return int(hashlib.sha256(value.encode()).hexdigest(), 16) % (10 ** 10)
    
    # Apply the consistent hash function to each combined string
    shipping_method_ids = combined_strings.apply(consistent_hash)
    
    return shipping_method_ids



def get_date_as_int(df, date_columns):
    """
    Converts the specified date columns from a DataFrame into integer keys
    in the mmddyyyy format.

    Args:
    df (pd.DataFrame): The input DataFrame containing the date columns.
    date_columns (list): A list of column names to be converted.

    Returns:
    pd.DataFrame: A DataFrame with date columns converted to integer mmddyyyy format.
    """
    for col in date_columns:
        # Convert the column to datetime to ensure correct format
        df[col] = pd.to_datetime(df[col], errors='coerce', format='%m/%d/%Y')

        # Convert to string in mm/dd/yyyy format
        df[col] = df[col].dt.strftime('%m%d%Y')

        # Convert to integer for keys
        df[col] = df[col].astype(int, errors='ignore')
    
    return df

def drop_columns(df, columns_to_drop):
    """
    Drops the specified columns from the DataFrame.
    
    Args:
    df (pd.DataFrame): The input DataFrame.
    columns_to_drop (list): A list of column names (strings) to be dropped from the DataFrame.
    
    Returns:
    pd.DataFrame: The DataFrame with the specified columns dropped.
    """
    # Drop the columns if they are found in the DataFrame
    df_dropped = df.drop(columns=[col for col in columns_to_drop if col in df.columns], axis=1)
    
    return df_dropped

# Keep the first occurrence of each 'Customer Id' (or 'Product Card Id')
dim_customer = dim_customer.groupby('Customer Id').first().reset_index()
dim_product = dim_product.groupby('Product Card Id').first().reset_index()

# Generate Region Ids and insert into dim_region
dim_region['Region Id'] = generate_region_id(dim_region)
dim_region.insert(0, 'Region Id', dim_region.pop('Region Id'))

# Generate dimension date
dim_date = create_date_dimension(Fact_Sales, date_columns)

# Step 1: Merge dim_region with Fact_Sales to include Region Id
Fact_Sales = Fact_Sales.merge(dim_region, how='left',
                              on=['Market', 'Order Region', 'Order Country', 'Order State', 'Order City'])

Fact_Sales = get_date_as_int(Fact_Sales, date_columns)

# Create a copy of dim_shipping for manipulation
dim_shipping1 = dim_shipping.copy()

# Drop duplicates and generate Shipping Method Id
dim_shipping1.drop_duplicates(subset=['Days for shipment (scheduled)', 'Shipping Mode'], inplace=True)
dim_shipping1['Shipping Method Id'] = generate_shipping_method_id(dim_shipping1)
dim_shipping1.insert(0, 'Shipping Method Id', dim_shipping1.pop('Shipping Method Id'))

# Merge Shipping Method Id back into Fact_Sales
Fact_Sales = Fact_Sales.merge(dim_shipping1, on=['Days for shipment (scheduled)', 'Shipping Mode'], how='left')

# Drop unnecessary columns
Fact_Sales = drop_columns(Fact_Sales, cols_to_drop)
Fact_Sales = Fact_Sales[new_cols_order]

# Step 3: Export to CSV
dim_customer.to_csv(os.path.join(processed_data_path, 'dim_customer.csv'), index=False)
dim_product.to_csv(os.path.join(processed_data_path, 'dim_product.csv'), index=False)
dim_date.to_csv(os.path.join(processed_data_path, 'dim_date.csv'), index=False)
dim_shipping1.to_csv(os.path.join(processed_data_path, 'dim_shipping.csv'), index=False)
dim_region.to_csv(os.path.join(processed_data_path, 'dim_region.csv'), index=False)
Fact_Sales.to_csv(os.path.join(processed_data_path, 'Fact_Sales.csv'), index=False)

print("CSV files have been exported successfully!")

CSV files have been exported successfully!
